# NER to Wikidata for coordinate retrieval

In [1]:
import os
import re
from lxml import etree
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, CSV
import sparql_dataframe
import requests
import uuid
import time
from tqdm import tqdm
import os
import pickle
from collections import Counter
from shapely.wkt import loads
import pydeck as pdk
import shapely.wkt


## Extracting XML elements

In [ ]:
def extract_elements(xml_file):
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
    with open(xml_file, 'rb') as file:
        tree = etree.parse(file)
    
    elements = {
        'placeName': tree.xpath('/tei:TEI/tei:text/tei:body/tei:sp/tei:ab/tei:seg/tei:reg/tei:placeName', namespaces=namespaces),
        'persName': tree.xpath('/tei:TEI/tei:text/tei:body/tei:sp/tei:ab/tei:seg/tei:reg/tei:persName', namespaces=namespaces),
        'target': tree.xpath('//tei:ptr/@target', namespaces=namespaces),
        'author': tree.xpath('//tei:author/tei:persName/text()', namespaces=namespaces),
        'title': tree.xpath('//tei:title/text()', namespaces=namespaces),
        'pubPlace': tree.xpath('//tei:pubPlace/text()', namespaces=namespaces),
        'date': tree.xpath('//tei:TEI/tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl/tei:date/@when', namespaces=namespaces)
    }
    
    return tree, elements

In [ ]:
def process_xml_folder(folder_path):
    data_place = []
    data_pers = []
    
    for filename in tqdm(os.listdir(folder_path), desc="Processing XML files"):
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_path, filename)
            tree, elements = extract_elements(file_path)
            
            placeNames = [el.text for el in elements['placeName']]
            persNames = [el.text for el in elements['persName']]
            target = elements['target'][0] if elements['target'] else None
            author = elements['author'][0] if elements['author'] else None
            title = elements['title'][0] if elements['title'] else None
            pubPlace = elements['pubPlace'][0] if elements['pubPlace'] else None
            date = elements['date'][0] if elements['date'] else None
            
            placeName_counts = Counter(placeNames)
            persName_counts = Counter(persNames)
            
            for placeName in set(placeNames):
                data_place.append({
                    'target': target,
                    'title': title,
                    'pubPlace': pubPlace,
                    'date': date,
                    'author': author,
                    'place': placeName,
                    'placeName_num': placeName_counts[placeName]
                })
                
            for persName in set(persNames):
                data_pers.append({
                    'target': target,
                    'title': title,
                    'pubPlace': pubPlace,
                    'date': date,
                    'author': author,
                    'person': persName,
                    'persName_num': persName_counts[persName]
                })
    
    return data_place, data_pers

In [ ]:
def create_dataframes(data_place, data_pers):
    df_place = pd.DataFrame(data_place)
    df_pers = pd.DataFrame(data_pers)
    
    df_place = df_place.drop_duplicates().sort_values(by='placeName_num', ascending=False)
    df_pers = df_pers.drop_duplicates().sort_values(by='persName_num', ascending=False)
    
    return df_place, df_pers

In [ ]:
folder_path = '/Users/nicola/Documents/Academia/Projects/TextEnt/Processing/NER'

data_place, data_pers = process_xml_folder(folder_path)
df_place, df_pers = create_dataframes(data_place, data_pers)

In [ ]:
df_place["place"] = df_place["place"].str.lower()
df_place.head(10)

In [ ]:
df_pers["person"] = df_pers["person"].str.lower()
df_pers.head()

In [ ]:
df_place_filter = df_place.groupby('target', group_keys=False).apply(lambda x: x.nlargest(2, 'placeName_num'))

In [ ]:
df_place_filter['uuid'] = df_place_filter['place'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_DNS, x)))

In [ ]:
df_place_filter

In [ ]:
#df_place.to_csv('/Users/carboni/Documents/Academia/Projects/TextEnt/output/df_place.csv', index=False)
#df_pers.to_csv('/Users/carboni/Documents/Academia/Projects/TextEnt/output/df_pers.csv', index=False)

### Removing Articles from NER results

In [ ]:
articles = [
    "l'",
    "le ",
    "la ",
    "les ",
    "un ",
    "une ",
    "des ",
    "du ",
    "de la ",
    "de l'",
    "de ",
    "d'",
    "au ",
    "aux ",
    "à ",
    "chez ",
    "sur ",
    "en ",
    "dans "
]

In [ ]:
articles = sorted(articles, key=len, reverse=True)

In [ ]:
def remove_article(place):
    # Check if the place starts with any of the articles
    for article in articles:
        if place.lower().startswith(article.lower()):  # Case-insensitive match
            return place[len(article):].strip()  # Remove the article and strip leading spaces
    return place  # Return the original if no article is found

In [ ]:
df_place_filter['place'] = df_place_filter['place'].apply(remove_article)

In [ ]:
print(df_place_filter[df_place_filter['uuid'].str.contains('9aebb788-d4a9-5959-b6ab-929f47c17c40')])

## Wikidata Queries

### Mythological Locations

In [ ]:
df_place_filter['place'].nunique()

In [ ]:
def query_sparql_for_mythological_places(
    df,
    place_column='place',
    target_column='target',
    uuid_column='uuid',
    endpoint_url='http://10.194.68.72:7001/sparql',
    output_columns=None
):
    """
    Queries the SPARQL endpoint for each unique place in the DataFrame and returns the results as a new DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing the places to query.
        place_column (str): The name of the column containing the place names.
        target_column (str): The name of the column containing the target values (to be included in the output).
        uuid_column (str): The name of the column containing the UUIDs (to be included in the output).
        endpoint_url (str): The URL of the SPARQL endpoint.
        output_columns (list): List of additional output columns to include. If None, defaults to extracting the usual fields.
        
    Returns:
        pd.DataFrame: A new DataFrame with the query results.
    """
    if output_columns is None:
        output_columns = [
            "place", "target", "uuid", "wikidata_id", "name", "coordinates",
            "typeLabel", "culture", "start_time", "end_time", "country",
            "countryLabels", "osm_id", "sitelinks", "roman_atlas_id",
            "pleiades_id", "topostext_id", "myths_id", "poleis_id", "manto_id"
        ]
    
    # SPARQL query template with placeholder for the place name
    query_template = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX schema: <http://schema.org/>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT ?a ?name ?coordinates ?typeLabel ?culture ?start_time ?end_time ?country (group_concat(?countryLabel; separator=";; ") AS ?countryLabels) ?osm_id ?sitelinks ?roman_atlas_id ?pleiades_id ?topostext_id ?myths_id ?poleis_id ?manto_id   
        WHERE {{
          {{
            SELECT ?a (MAX(?sitelinks) AS ?maxSitelinks)
            WHERE {{
              VALUES ?category {{ wd:Q3238337 }} # only mythical location    
              ?a (wdt:P31)/((wdt:P279)*) ?category.
              ?a rdfs:label ?name .
              ?a ^schema:about/wikibase:sitelinks ?sitelinks .

              FILTER (LANG(?name) = "fr" || LANG(?name) = "en") .
              FILTER REGEX(STR(?name), "{value}", "i") .
            }}
            GROUP BY ?a ?name
            ORDER BY DESC(?maxSitelinks)
            LIMIT 1
          }}

          ?a rdfs:label ?name .
          ?a wdt:P31 ?type .
          ?a wdt:P625 ?coordinates . 
          ?a ^schema:about/wikibase:sitelinks ?sitelinks .

          ?type rdfs:label ?typeLabel .

          OPTIONAL {{
            ?a wdt:P361 ?partOf .
            ?partOf rdfs:label ?partOfLabel .
            FILTER (LANG(?partOfLabel) = "en") .
          }}

          OPTIONAL {{
            ?a wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER (LANG(?countryLabel) = "en") .
          }}

        OPTIONAL {{
             ?a wdt:P2596 ?culture . 
             ?culture wdt:P580 ?start_time .
             ?culture wdt:P582 ?end_time .
          }} 

        OPTIONAL {{
            ?a wdt:P1584 ?pleiades_id
          }}

         OPTIONAL {{
            ?a wdt:P402 ?osm_id
          }}

        OPTIONAL {{
            ?a wdt:P8068 ?topostext_id . 
          }}
        OPTIONAL {{
            ?a wdt:P361 ?partOf
          }}
        OPTIONAL {{
            ?a wdt:P1936 ?roman_atlas_id . 
          }}  
        OPTIONAL {{
            ?a wdt:P12402 ?myths_id . 
          }}  
        OPTIONAL {{
            ?a wdt:P8137 ?poleis_id .
          }}
        OPTIONAL {{
            ?a wdt:P9736 ?manto_id .
          }} 

          FILTER (LANG(?name) = "en") .
          FILTER (LANG(?typeLabel) = "en") .
        }}
        GROUP BY ?a ?name ?coordinates ?typeLabel ?culture ?start_time ?end_time ?country ?sitelinks ?osm_id ?roman_atlas_id ?pleiades_id ?topostext_id ?myths_id ?poleis_id ?manto_id
        ORDER BY DESC(?sitelinks)
    """
    
    results = []
    
    # Deduplicate the places to avoid querying the same place twice
    unique_places = df[[place_column, target_column, uuid_column]].drop_duplicates(subset=[place_column])
    
    # Iterate over the DataFrame and query for each unique place
    for _, row in tqdm(unique_places.iterrows(), total=len(unique_places), desc="Querying SPARQL Endpoint"):
        place_name = row[place_column]
        target = row[target_column]
        uuid = row[uuid_column]

        # Replace {value} with the place name
        query = query_template.format(value=place_name)

        try:
            # Send the request to the SPARQL endpoint
            response = requests.get(endpoint_url, params={'query': query, 'format': 'json'})
            if response.status_code == 200:
                query_result = response.json().get("results", {}).get("bindings", [])
                
                if query_result:
                    for result in query_result:
                        # Extract data from the JSON response
                        place_data = {
                            "place": place_name,
                            "target": target,
                            "uuid": uuid,
                            "wikidata_id": result.get("a", {}).get("value", ""),
                            "name": result.get("name", {}).get("value", ""),
                            "coordinates": result.get("coordinates", {}).get("value", ""),
                            "typeLabel": result.get("typeLabel", {}).get("value", ""),
                            "culture": result.get("culture", {}).get("value", ""),
                            "start_time": result.get("start_time", {}).get("value", ""),
                            "end_time": result.get("end_time", {}).get("value", ""),
                            "country": result.get("country", {}).get("value", ""),
                            "countryLabels": result.get("countryLabels", {}).get("value", ""),
                            "osm_id": result.get("osm_id", {}).get("value", ""),
                            "sitelinks": result.get("sitelinks", {}).get("value", ""),
                            "roman_atlas_id": result.get("roman_atlas_id", {}).get("value", ""),
                            "pleiades_id": result.get("pleiades_id", {}).get("value", ""),
                            "topostext_id": result.get("topostext_id", {}).get("value", ""),
                            "myths_id": result.get("myths_id", {}).get("value", ""),
                            "poleis_id": result.get("poleis_id", {}).get("value", ""),
                            "manto_id": result.get("manto_id", {}).get("value", "")
                        }
                        results.append(place_data)
                else:
                    print(f"No data found for place: {place_name}")
            else:
                print(f"Failed to retrieve data for place: {place_name}, Status code: {response.status_code}")
        except requests.RequestException as e:
            print(f"Error querying API for place: {place_name}, Error: {e}")

        # Wait for 1 second to avoid overloading the server
        time.sleep(1)
    
    # Convert the list of results into a DataFrame
    place_mythological_only = pd.DataFrame(results, columns=output_columns)
    
    return place_mythological_only

In [ ]:
place_mythological_only = query_sparql_for_mythological_places(df_place_filter, place_column='place', target_column='target', uuid_column='uuid')

### Historical Locations

### All other Locations

In [ ]:
def save_progress(results, filename='sparql_results_temp.csv'):
    temp_df = pd.DataFrame(results)
    temp_df.to_csv(filename, index=False)

def load_progress(filename='sparql_results_temp.csv'):
    if os.path.exists(filename):
        temp_df = pd.read_csv(filename)
        return temp_df.to_dict('records')
    return []

def load_query_cache(filename='query_cache.pkl'):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    return {}

def save_query_cache(cache, filename='query_cache.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(cache, f)

In [ ]:
def query_wikidata(place):
    if place in query_cache:
        return query_cache[place]

    query_template = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX schema: <http://schema.org/>

    SELECT ?a ?name ?coordinates ?typeLabel ?culture ?start_time ?end_time ?country (group_concat(?countryLabel; separator=";; ") AS ?countryLabels) ?osm_id ?sitelinks ?roman_atlas_id ?pleiades_id ?topostext_id ?myths_id ?poleis_id ?manto_id   
    WHERE {{
      {{
        SELECT ?a (MAX(?sitelinks) AS ?maxSitelinks)
        WHERE {{
          VALUES ?category {{ wd:Q6256 wd:Q82794 wd:Q1620908 }}
          ?a (wdt:P31)/((wdt:P279)*) ?category.
          ?a rdfs:label ?name .
          ?a ^schema:about/wikibase:sitelinks ?sitelinks .

          FILTER (LANG(?name) = "fr") .
          FILTER REGEX(STR(?name), "^{place}$", "i") .
        }}
        GROUP BY ?a
        ORDER BY DESC(?maxSitelinks)
        LIMIT 1
      }}

      ?a rdfs:label ?name .
      ?a wdt:P31 ?type .
      ?a wdt:P625 ?coordinates . 
      ?a ^schema:about/wikibase:sitelinks ?sitelinks .
     
      ?type rdfs:label ?typeLabel .
      
      
      OPTIONAL {{
        ?a wdt:P17 ?country .
        ?country rdfs:label ?countryLabel .
        FILTER (LANG(?countryLabel) = "en") .
      }}
      
    OPTIONAL {{
         ?a wdt:P2596 ?culture . 
         ?culture wdt:P580 ?start_time .
         ?culture wdt:P582 ?end_time .
      }} 
      
    OPTIONAL {{
        ?a wdt:P1584 ?pleiades_id
      }}

     OPTIONAL {{
        ?a wdt:P402 ?osm_id
      }}
      
    OPTIONAL {{
        ?a wdt:P8068 ?topostext_id . 
      }}
    OPTIONAL {{
        ?a wdt:P361 ?partOf
      }}
    OPTIONAL {{
        ?a wdt:P1936 ?roman_atlas_id . 
      }}  
    OPTIONAL {{
        ?a wdt:P12402 ?myths_id . 
      }}  
    OPTIONAL {{
        ?a wdt:P8137 ?poleis_id .
      }}
    OPTIONAL {{
        ?a wdt:P9736 ?manto_id .
      }} 

      FILTER (LANG(?name) = "en") .
      FILTER (LANG(?typeLabel) = "en") .
    }}
    GROUP BY ?a ?name ?coordinates ?typeLabel ?culture ?start_time ?end_time ?country ?sitelinks ?osm_id ?roman_atlas_id ?pleiades_id ?topostext_id ?myths_id ?poleis_id ?manto_id
    ORDER BY DESC(?sitelinks)
    """
    
    query = query_template.format(place=place)
    #url = 'https://qlever.cs.uni-freiburg.de/api/wikidata'
    url = 'http://10.194.68.72:7001' #internal unige
    response = requests.get(url, params={'query': query, 'output': 'json'})
    
    if response.status_code == 200:
        raw_results = response.json().get('results', {}).get('bindings', [])
        # Process results to ensure all values are strings
        processed_results = []
        for result in raw_results:
            processed_result = {key: str(value.get('value', '')) for key, value in result.items()}
            processed_results.append(processed_result)
        query_cache[place] = processed_results
        return processed_results
    else:
        query_cache[place] = None  # Cache failed queries as well
        return None

In [ ]:
# Initialize or load query cache
query_cache = load_query_cache()

# Load existing results if any
results = load_progress()

# Get the set of already processed places
processed_places = set([entry['place'] for entry in results])

# Extract unique places from your DataFrame
unique_places_df = df_place_filter.drop_duplicates(subset='place')

# Initialize a counter for saving progress periodically
counter = 0
save_every_n = 10  # Save progress every N iterations

# Iterate over unique places
for index, row in tqdm(unique_places_df.iterrows(), total=len(unique_places_df), desc="Querying QLever"):
    place = row['place']
    target = row['target']
    uuid = row['uuid']
    
    if place in processed_places:
        continue
    
    sparql_results = query_wikidata(place)
    
    if sparql_results:
        for result in sparql_results:
            results.append({
                'place': place,
                'target': target,
                'uuid': uuid,
                'wikidata_id': result.get('a', ''),
                'name': result.get('name', ''),
                'coordinates': result.get('coordinates', ''),
                'typeLabel': result.get('typeLabel', ''),
                'country': result.get('country', ''),
                'countryLabel': result.get('countryLabels', ''),
                'culture': result.get('culture', ''),
                'start_time': result.get('start_time', ''),
                'end_time': result.get('end_time', ''),
                'partOf': result.get('partOf', ''),
                'sitelinks': result.get('sitelinks', ''),
                'osm_id': result.get('osm_id', ''),
                'roman_atlas_id': result.get('roman_atlas_id', ''),
                'pleiades_id': result.get('pleiades_id', ''),
                'topostext_id': result.get('topostext_id', ''),
                'myths_id': result.get('myths_id', ''),
                'poleis_id': result.get('poleis_id', ''),
                'manto_id': result.get('manto_id', '')
            })
    else:
        # Handle cases where there is no result
        results.append({
            'place': place,
            'uuid': uuid,
            'target': target,
            'wikidata_id': '',
            'name': '',
            'coordinates': '',
            'typeLabel': '',
            'country': '',
            'countryLabels': '',
            'culture': '',
            'start_time': '',
            'end_time': '',
            'partOf': '',
            'sitelinks': '',
            'osm_id': '',
            'roman_atlas_id': '',
            'pleiades_id': '',
            'topostext_id': '',
            'myths_id': '',
            'poleis_id': '',
            'manto_id': ''
        })
    
    processed_places.add(place)
    counter += 1
    
    # Save progress and cache every N iterations
    if counter % save_every_n == 0:
        save_progress(results)
        save_query_cache(query_cache)
        # Optionally, print a message
        # print(f"Saved progress after processing {counter} places.")
    
    # Optional: Remove or adjust sleep time if necessary
    time.sleep(1)

# After processing all places, save final progress and cache
save_progress(results)
save_query_cache(query_cache)

# Convert the results to a new DataFrame
df_place_coordinates = pd.DataFrame(results)

In [ ]:
#!rm query_cache.pkl
#!rm sparql_results_temp.csv

In [1]:
df_place_coordinates

NameError: name 'df_place_coordinates' is not defined

### Concat classes and labels

aggregate instead of using SPARQL concat

In [ ]:
concat_cols = ['typeLabel', 'countryLabel']

In [ ]:
def get_agg_func(col):
    if col in concat_cols:
        return lambda x: ';;'.join(map(str, x.unique()))
    else:
        return 'first'

In [ ]:
agg_dict = {col: get_agg_func(col) for col in df_place_coordinates.columns if col != 'uuid'}

In [ ]:
df_place_concat = df_place_coordinates.groupby('uuid', as_index=False).agg(agg_dict)

In [ ]:
df_place_concat

In [ ]:
#df_place_concat.to_csv('df_place_coordinates.csv', index=False)

## Query OSM for WKT Multipolygon (filtering labelType)

In [ ]:
filter_types = ['river', 'lake', 'region']

In [ ]:
mask = df_place_concat['typeLabel'].str.contains('|'.join(filter_types), case=False, na=False)

In [ ]:
mask = mask & df_place_concat['osm_id'].notnull()
df_filtered = df_place_concat[mask].copy()

In [ ]:
def get_osm_wkt(osm_id):
    # Ensure osm_id is a string (in case it's not)
    osm_id_str = str(osm_id).strip()
    
    # Build the SPARQL query, substituting the osm_id
    query_template = """
    PREFIX ogc: <http://www.opengis.net/rdf#>
    PREFIX osmrel: <https://www.openstreetmap.org/relation/>
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX osm: <https://www.openstreetmap.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX osmkey: <https://www.openstreetmap.org/wiki/Key:>
    SELECT ?shape WHERE {{
      osmrel:{osm_id} geo:hasGeometry/geo:asWKT ?shape
    }}
    """
    query = query_template.format(osm_id=osm_id_str)
    url = 'https://qlever.cs.uni-freiburg.de/api/osm-planet'

    # Send the request
    try:
        response = requests.get(url, params={'query': query, 'output': 'json'})
        if response.status_code == 200:
            results = response.json().get('results', {}).get('bindings', [])
            if results:
                # Get the 'shape' value
                shape = results[0].get('shape', {}).get('value', '')
                return shape
            else:
                # No results found
                return None
        else:
            print(f"Error querying osm_id {osm_id}: HTTP {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception querying osm_id {osm_id}: {e}")
        return None

In [ ]:
def fetch_osm_wkt(row):
    return get_osm_wkt(row['osm_id'])

In [ ]:
df_filtered['osm_wkt'] = df_filtered.apply(fetch_osm_wkt, axis=1)

In [ ]:
df_filtered.head()

In [ ]:
df_place_concat['osm_wkt'] = None
df_place_concat.loc[df_filtered.index, 'osm_wkt'] = df_filtered['osm_wkt']

In [ ]:
df_place_concat.to_csv('df_place_coordinates.csv', index=False)

## Query WHG

#### Further Refiniment

if it is a river or a waterway, get the coordinate from OpenStreetMap. Possible to do using the instance in QLever (sparql = https://qlever.cs.uni-freiburg.de/api/osm-planet
)
```
PREFIX ogc: <http://www.opengis.net/rdf#>
PREFIX osmrel: <https://www.openstreetmap.org/relation/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX osm: <https://www.openstreetmap.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX osmkey: <https://www.openstreetmap.org/wiki/Key:>
SELECT * WHERE {
  osmrel:2188548 geo:hasGeometry/geo:asWKT ?shape
}
```
problems: somehow throgh python it does not retrieve the type

if results are empty, query TGN. Althought it is quite slow:

```
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?place ?geojson ?typeLabel WHERE {
  ?place a crm:E53_Place .
  ?place rdfs:label "oristano" .
  ?place crm:P1_is_identified_by ?geometry .
  ?geometry crm:P2_has_type <http://geojson.org> ;
            crm:P90_has_value ?geojson .
  ?place crm:P2_has_type ?type .
  ?type rdfs:label ?typeLabel .
} LIMIT 100
```

Other option, use world historical gazetteer API
https://whgazetteer.org/api/index/?name=oristano

In [ ]:
def query_whg_api_for_empty_coordinates(df, place_column='place', coordinates_column='coordinates', log_file='result_whg.txt'):
    """
    Queries the WHG API for places where the coordinates are missing, retrieves longitude and latitude,
    and stores them as a POINT string directly in the existing coordinates column.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        place_column (str): The name of the column containing the place names to query.
        coordinates_column (str): The name of the column where coordinates are missing and to store the POINT string.
        log_file (str): The file where messages for places with no features found will be logged.
        
    Returns:
        pd.DataFrame: The DataFrame with the updated coordinates column containing the POINT string.
    """
    # Check which rows have empty 'coordinates'
    empty_coordinates_mask = df[coordinates_column].isna() | (df[coordinates_column] == '')
    
    # Get rows with empty 'coordinates' for iteration
    rows_to_query = df[empty_coordinates_mask]
    
    # Open the log file to write messages
    with open(log_file, 'a') as log:
        # Iterate over rows with empty 'coordinates' using tqdm for progress bar
        for index, row in tqdm(rows_to_query.iterrows(), total=len(rows_to_query), desc="Querying WHG API"):
            place_name = row[place_column]

            # Construct the API URL
            api_url = f"https://whgazetteer.org/api/index/?name={place_name}"

            try:
                # Make the API request
                response = requests.get(api_url)

                # Check if the request was successful (status code 200)
                if response.status_code == 200:
                    data = response.json()
                    
                    # Extract the coordinates using the JSON path
                    if 'features' in data and len(data['features']) > 0:
                        geometry = data['features'][0].get('geometry', None)
                        if geometry and 'coordinates' in geometry:
                            coordinates = geometry['coordinates']
                            if len(coordinates) >= 2:
                                longitude = coordinates[0]
                                latitude = coordinates[1]
                                
                                # Create the POINT string
                                point_str = f"POINT({longitude} {latitude})"
                                df.at[index, coordinates_column] = point_str
                            else:
                                log.write(f"Coordinates not found for place: {place_name}\n")
                                df.at[index, coordinates_column] = None
                        else:
                            log.write(f"No valid geometry found for place: {place_name}\n")
                            df.at[index, coordinates_column] = None
                    else:
                        log.write(f"No features found for place: {place_name}\n")
                        df.at[index, coordinates_column] = None
                else:
                    print(f"Failed to retrieve data for place: {place_name}, Status code: {response.status_code}")
                    df.at[index, coordinates_column] = None  # Save None if the request failed
            except requests.RequestException as e:
                print(f"Error querying API for place: {place_name}, Error: {e}")
                df.at[index, coordinates_column] = None  # Save None if there was an error

            # Wait for 2 seconds before the next request
            time.sleep(2)
    
    return df

In [ ]:
df_place_concat = query_whg_api_for_empty_coordinates(df_place_concat)

In [ ]:
df_place_concat

In [ ]:
df_place_concat

## Map Creation

In [ ]:
df_place_concat = df_place_concat.dropna(subset=['coordinates'])

In [ ]:
df_place_concat['place_number'] = df_place_concat.groupby(['place', 'country'])['place'].transform('count')

In [ ]:
df_place_concat = df_place_concat[df_place_concat['coordinates'] != "bn860104"]

In [ ]:
df_place_concat['geometry'] = df_place_concat['coordinates'].apply(loads)

In [ ]:
df_place_concat['latitude'] = df_place_concat['geometry'].apply(lambda geom: geom.y)
df_place_concat['longitude'] = df_place_concat['geometry'].apply(lambda geom: geom.x)

In [ ]:
df_no_duplicates = df_place_concat.drop_duplicates(subset=['place', 'country'])

In [ ]:
df_no_duplicates

In [ ]:
df_no_duplicates = df_no_duplicates.drop(columns=['geometry'])

In [ ]:
scatter_layer = pdk.Layer(
    'ScatterplotLayer',
    df_no_duplicates,
    opacity=0.6,
    get_position=['longitude', 'latitude'],
    get_radius='place_number * 5000',
    get_fill_color=[255, 0, 0],  # Red 
    pickable=True,
    stroked=True,
    get_line_color=[255,255,255]
)

In [ ]:
view_state = pdk.ViewState(
    latitude=df_no_duplicates['latitude'].mean(),
    longitude=df_no_duplicates['longitude'].mean(),
    zoom=3,
)

In [ ]:
tooltip = {
    "html": "<b>{place_number}</b> reference to <b>{place}</b>",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

In [ ]:
deck = pdk.Deck(
    layers=[scatter_layer],
    initial_view_state=view_state,
    tooltip=tooltip,
    map_provider="carto",
    map_style="light" #possible here to go for light’, ‘dark’, ‘road’, ‘satellite’, 
    #‘dark_no_labels’, and ‘light_no_labels’. Also possible to use mapbox. To change together with the 
    #parameters on scatter_layer (e.g. opacity!)
)

In [ ]:
deck.to_html(filename='textent_map.html', offline=True, open_browser=False, notebook_display=False)